# Visuals for LDIC2022 Paper

Here, the visuals for the publication are created.

Import standard libraries.

In [ ]:
# Python standard libraries
import os
import string
import pathlib
import datetime
import random

# scientific standard libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import gridspec
from matplotlib import patches as mpatches
from matplotlib import dates as mdates

# Jupyter notebook-related libraries
from IPython.display import Markdown
from IPython.display import Image

Import conflowgen

In [ ]:
from conflowgen import ContainerFlowGenerationManager
from conflowgen import ModeOfTransport
from conflowgen import PortCallManager
from conflowgen import ExportFileFormat
from conflowgen import ExportContainerFlowManager
from conflowgen import DatabaseChooser
from conflowgen import setup_logger
from conflowgen import InboundAndOutboundVehicleCapacityPreviewReport
from conflowgen import ContainerFlowByVehicleTypePreviewReport
from conflowgen import VehicleCapacityExceededPreviewReport
from conflowgen import ModalSplitPreviewReport
from conflowgen import InboundAndOutboundVehicleCapacityAnalysisReport
from conflowgen import ContainerFlowByVehicleTypeAnalysisReport
from conflowgen import ModalSplitAnalysisReport
from conflowgen import ContainerFlowAdjustmentByVehicleTypeAnalysisReport
from conflowgen import ContainerFlowAdjustmentByVehicleTypeAnalysisSummaryReport
from conflowgen import ContainerLengthDistributionManager
from conflowgen import ContainerLength
from conflowgen import ModeOfTransportDistributionManager

## Generate data

Now, conflowgen is initialized with the schedules.
This is based on the script `demo_DEHAM_CTA.py` but is shortened.
Some logging messages and some comments have been removed for brevity.
For further information, turn to `demo_DEHAM_CTA.py`.

In [ ]:
import conflowgen

seeded_random = random.Random(x=1)

import_root_dir = os.path.abspath(
    os.path.join(
        os.path.dirname(conflowgen.__path__[0]),
        "demo",
        "data"
    )
)
import_deham_dir = os.path.join(
    import_root_dir,
    "DEHAM",
    "CT Altenwerder"
)
df_deep_sea_vessels = pd.read_csv(
    os.path.join(
        import_deham_dir,
        "deep_sea_vessel_input.csv"
    ),
    index_col=[0]
)
df_feeders = pd.read_csv(
    os.path.join(
        import_deham_dir,
        "feeder_input.csv"
    ),
    index_col=[0]
)
df_barges = pd.read_csv(
    os.path.join(
        import_deham_dir,
        "barge_input.csv"
    ),
    index_col=[0]
)
df_trains = pd.read_csv(
    os.path.join(
        import_deham_dir,
        "train_input.csv"
    ),
    index_col=[0]
)

logger = setup_logger()
database_chooser = DatabaseChooser()


def initialize_conflowgen(figure_name: str, force_reload=False) -> ContainerFlowGenerationManager:
    demo_file_name = f"demo_deham_cta_visual_{figure_name}.sqlite"

    if demo_file_name in database_chooser.list_all_sqlite_databases():
        database_chooser.load_existing_sqlite_database(demo_file_name)
        if not force_reload:
            container_flow_generation_manager = ContainerFlowGenerationManager()
            return container_flow_generation_manager
    else:
        database_chooser.create_new_sqlite_database(demo_file_name)

    container_flow_generation_manager = ContainerFlowGenerationManager()
    container_flow_start_date = datetime.date(year=2021, month=7, day=1)
    container_flow_end_date = datetime.date(year=2021, month=7, day=31)
    container_flow_generation_manager.set_properties(
        name="Demo DEHAM CTA Visual 1a",
        start_date=container_flow_start_date,
        end_date=container_flow_end_date
    )
    port_call_manager = PortCallManager()
    for i, row in df_feeders.iterrows():
        feeder_vehicle_name = row["vehicle_name"] + "-unique"
        capacity = row["capacity"]
        vessel_arrives_at_as_pandas_type = row["arrival (planned)"]
        vessel_arrives_at_as_datetime_type = pd.to_datetime(vessel_arrives_at_as_pandas_type)

        if vessel_arrives_at_as_datetime_type.date() < container_flow_start_date:
            logger.info(f"Skipping feeder '{feeder_vehicle_name}' because it arrives before the start")
            continue

        if vessel_arrives_at_as_datetime_type.date() > container_flow_end_date:
            logger.info(f"Skipping feeder '{feeder_vehicle_name}' because it arrives after the end")
            continue

        if port_call_manager.get_schedule(feeder_vehicle_name, vehicle_type=ModeOfTransport.feeder):
            logger.info(f"Skipping feeder '{feeder_vehicle_name}' because it already exists")
            continue

        logger.info(f"Add feeder '{feeder_vehicle_name}' to database")
        moved_capacity = int(round(capacity * seeded_random.uniform(0.3, 0.8) / 2))
        port_call_manager.add_large_scheduled_vehicle(
            vehicle_type=ModeOfTransport.feeder,
            service_name=feeder_vehicle_name,
            vehicle_arrives_at=vessel_arrives_at_as_datetime_type.date(),
            vehicle_arrives_at_time=vessel_arrives_at_as_datetime_type.time(),
            average_vehicle_capacity=capacity,
            average_moved_capacity=moved_capacity,
            vehicle_arrives_every_k_days=-1
        )

    logger.info("Start importing deep sea vessels...")
    for i, row in df_deep_sea_vessels.iterrows():
        deep_sea_vessel_vehicle_name = row["vehicle_name"] + "-unique"
        capacity = row["capacity"]
        vessel_arrives_at_as_pandas_type = row["arrival (planned)"]
        vessel_arrives_at_as_datetime_type = pd.to_datetime(vessel_arrives_at_as_pandas_type)

        if vessel_arrives_at_as_datetime_type.date() < container_flow_start_date:
            logger.info(f"Skipping deep sea vessel '{deep_sea_vessel_vehicle_name}' because it arrives before the start")
            continue

        if vessel_arrives_at_as_datetime_type.date() > container_flow_end_date:
            logger.info(f"Skipping deep sea vessel '{deep_sea_vessel_vehicle_name}' because it arrives after the end")
            continue

        if port_call_manager.get_schedule(deep_sea_vessel_vehicle_name, vehicle_type=ModeOfTransport.deep_sea_vessel):
            logger.info(f"Skipping deep sea service '{deep_sea_vessel_vehicle_name}' because it already exists")
            continue

        logger.info(f"Add deep sea vessel '{deep_sea_vessel_vehicle_name}' to database")
        moved_capacity = int(round(capacity * seeded_random.uniform(0.25, 0.5) / 2))
        port_call_manager.add_large_scheduled_vehicle(
            vehicle_type=ModeOfTransport.deep_sea_vessel,
            service_name=deep_sea_vessel_vehicle_name,
            vehicle_arrives_at=vessel_arrives_at_as_datetime_type.date(),
            vehicle_arrives_at_time=vessel_arrives_at_as_datetime_type.time(),
            average_vehicle_capacity=capacity,
            average_moved_capacity=moved_capacity,
            vehicle_arrives_every_k_days=-1
        )

    logger.info("Start importing barges...")
    for i, row in df_barges.iterrows():
        barge_vehicle_name = row["vehicle_name"] + "-unique"
        capacity = row["capacity"]
        vessel_arrives_at_as_pandas_type = row["arrival (planned)"]
        vessel_arrives_at_as_datetime_type = pd.to_datetime(vessel_arrives_at_as_pandas_type)

        if vessel_arrives_at_as_datetime_type.date() < container_flow_start_date:
            logger.info(f"Skipping barge '{barge_vehicle_name}' because it arrives before the start")
            continue

        if vessel_arrives_at_as_datetime_type.date() > container_flow_end_date:
            logger.info(f"Skipping barge '{barge_vehicle_name}' because it arrives after the end")
            continue

        if port_call_manager.get_schedule(barge_vehicle_name, vehicle_type=ModeOfTransport.barge):
            logger.info(f"Skipping barge '{barge_vehicle_name}' because it already exists")
            continue

        logger.info(f"Add barge '{barge_vehicle_name}' to database")
        moved_capacity = int(round(capacity * seeded_random.uniform(0.3, 0.6)))
        port_call_manager.add_large_scheduled_vehicle(
            vehicle_type=ModeOfTransport.barge,
            service_name=barge_vehicle_name,
            vehicle_arrives_at=vessel_arrives_at_as_datetime_type.date(),
            vehicle_arrives_at_time=vessel_arrives_at_as_datetime_type.time(),
            average_vehicle_capacity=capacity,
            average_moved_capacity=moved_capacity,
            vehicle_arrives_every_k_days=-1
        )

    logger.info("Start importing trains...")
    for i, row in df_trains.iterrows():
        train_vehicle_name = row["vehicle_name"]
        vessel_arrives_at_as_pandas_type = row["arrival_day"]
        vessel_arrives_at_as_datetime_type = pd.to_datetime(vessel_arrives_at_as_pandas_type)

        if port_call_manager.get_schedule(train_vehicle_name, vehicle_type=ModeOfTransport.train):
            logger.info(f"Train service '{train_vehicle_name}' already exists")
            continue

        capacity = 96  # in TEU, see https://www.intermodal-info.com/verkehrstraeger/
        earliest_time = datetime.time(hour=1, minute=0)
        earliest_time_as_delta = datetime.timedelta(hours=earliest_time.hour, minutes=earliest_time.minute)
        latest_time = datetime.time(hour=5, minute=30)
        latest_time_as_delta = datetime.timedelta(hours=latest_time.hour, minutes=latest_time.minute)
        number_slots_minus_one = int((latest_time_as_delta - earliest_time_as_delta) / datetime.timedelta(minutes=30))

        drawn_slot = seeded_random.randint(0, number_slots_minus_one)
        vehicle_arrives_at_time_as_delta = earliest_time_as_delta + datetime.timedelta(hours=0.5 * drawn_slot)
        vehicle_arrives_at_time = (datetime.datetime.min + vehicle_arrives_at_time_as_delta).time()
        logger.info(f"Add train '{train_vehicle_name}' to database")
        port_call_manager.add_large_scheduled_vehicle(
            vehicle_type=ModeOfTransport.train,
            service_name=train_vehicle_name,
            vehicle_arrives_at=vessel_arrives_at_as_datetime_type.date(),
            vehicle_arrives_at_time=vehicle_arrives_at_time,
            average_vehicle_capacity=capacity,
            average_moved_capacity=capacity,
            vehicle_arrives_every_k_days=7
        )

        return container_flow_generation_manager

In [ ]:
# In this scenario, we move traffic away from feeders to deep sea vessels

container_flow_generation_manager = initialize_conflowgen("1a")

diff = 0.1

mode_of_transport_distribution_1a = {
    ModeOfTransport.truck: {
        ModeOfTransport.truck: 0,
        ModeOfTransport.train: 0,
        ModeOfTransport.barge: 0,
        ModeOfTransport.feeder: 0.8 / (0.8 + 4.6) - diff,
        ModeOfTransport.deep_sea_vessel: 4.6 / (0.8 + 4.6) + diff
    },
    ModeOfTransport.train: {
        ModeOfTransport.truck: 0,
        ModeOfTransport.train: 0,
        ModeOfTransport.barge: 0,
        ModeOfTransport.feeder: 0.8 / (0.8 + 4.6) - diff,
        ModeOfTransport.deep_sea_vessel: 4.6 / (0.8 + 4.6) + diff
    },
    ModeOfTransport.barge: {
        ModeOfTransport.truck: 0,
        ModeOfTransport.train: 0,
        ModeOfTransport.barge: 0,
        ModeOfTransport.feeder: 0.8 / (0.8 + 4.6) - diff,
        ModeOfTransport.deep_sea_vessel: 4.6 / (0.8 + 4.6) + diff
    },
    ModeOfTransport.feeder: {
        ModeOfTransport.truck: 0.8 / (0.8 + 1.9) * 0.502,
        ModeOfTransport.train: 0.8 / (0.8 + 1.9) * 0.47,
        ModeOfTransport.barge: 0.8 / (0.8 + 1.9) * 0.0028,
        ModeOfTransport.feeder: 0,
        ModeOfTransport.deep_sea_vessel: 1.9 / (0.8 + 1.9)
    },
    ModeOfTransport.deep_sea_vessel: {
        ModeOfTransport.truck: 4.6 / (4.6 + 1.9) * 0.502,
        ModeOfTransport.train: 4.6 / (4.6 + 1.9) * 0.47,
        ModeOfTransport.barge: 4.6 / (4.6 + 1.9) * 0.0028,
        ModeOfTransport.feeder: 1.9 / (4.6 + 1.9),
        ModeOfTransport.deep_sea_vessel: 0
    }
}

ModeOfTransportDistributionManager().set_mode_of_transport_distributions(mode_of_transport_distribution_1a)

container_flow_generation_manager.generate()
export_container_flow_manager = ExportContainerFlowManager()
folder_data_visual_1a = "demo-DEHAM-visual-1a-0-1--" + str(datetime.datetime.now()).replace(":", "-").replace(" ", "--").split(".")[0]
export_container_flow_manager.export(
    folder_name=folder_data_visual_1a,
    file_format=ExportFileFormat.csv
)

database_chooser.close_current_connection()

In [ ]:
# In this scenario, we keep things as they are

container_flow_generation_manager = initialize_conflowgen("1b")

mode_of_transport_distribution_1b = {
    ModeOfTransport.truck: {
        ModeOfTransport.truck: 0,
        ModeOfTransport.train: 0,
        ModeOfTransport.barge: 0,
        ModeOfTransport.feeder: 0.8 / (0.8 + 4.6),
        ModeOfTransport.deep_sea_vessel: 4.6 / (0.8 + 4.6)
    },
    ModeOfTransport.train: {
        ModeOfTransport.truck: 0,
        ModeOfTransport.train: 0,
        ModeOfTransport.barge: 0,
        ModeOfTransport.feeder: 0.8 / (0.8 + 4.6),
        ModeOfTransport.deep_sea_vessel: 4.6 / (0.8 + 4.6)
    },
    ModeOfTransport.barge: {
        ModeOfTransport.truck: 0,
        ModeOfTransport.train: 0,
        ModeOfTransport.barge: 0,
        ModeOfTransport.feeder: 0.8 / (0.8 + 4.6),
        ModeOfTransport.deep_sea_vessel: 4.6 / (0.8 + 4.6)
    },
    ModeOfTransport.feeder: {
        ModeOfTransport.truck: 0.8 / (0.8 + 1.9) * 0.502,
        ModeOfTransport.train: 0.8 / (0.8 + 1.9) * 0.47,
        ModeOfTransport.barge: 0.8 / (0.8 + 1.9) * 0.0028,
        ModeOfTransport.feeder: 0,
        ModeOfTransport.deep_sea_vessel: 1.9 / (0.8 + 1.9)
    },
    ModeOfTransport.deep_sea_vessel: {
        ModeOfTransport.truck: 4.6 / (4.6 + 1.9) * 0.502,
        ModeOfTransport.train: 4.6 / (4.6 + 1.9) * 0.47,
        ModeOfTransport.barge: 4.6 / (4.6 + 1.9) * 0.0028,
        ModeOfTransport.feeder: 1.9 / (4.6 + 1.9),
        ModeOfTransport.deep_sea_vessel: 0
    }
}

ModeOfTransportDistributionManager().set_mode_of_transport_distributions(mode_of_transport_distribution_1b)

container_flow_generation_manager.generate()
export_container_flow_manager = ExportContainerFlowManager()
folder_data_visual_1b = "demo-DEHAM-visual-1b-0-1--" + str(datetime.datetime.now()).replace(":", "-").replace(" ", "--").split(".")[0]
export_container_flow_manager.export(
    folder_name=folder_data_visual_1b,
    file_format=ExportFileFormat.csv
)

database_chooser.close_current_connection()

In [ ]:
container_flow_generation_manager = initialize_conflowgen("1c")

# In this scenario, we move traffic away from deep sea vessels to feeders
mode_of_transport_distribution_1c = {
    ModeOfTransport.truck: {
        ModeOfTransport.truck: 0,
        ModeOfTransport.train: 0,
        ModeOfTransport.barge: 0,
        ModeOfTransport.feeder: 0.8 / (0.8 + 4.6) + diff,
        ModeOfTransport.deep_sea_vessel: 4.6 / (0.8 + 4.6) - diff
    },
    ModeOfTransport.train: {
        ModeOfTransport.truck: 0,
        ModeOfTransport.train: 0,
        ModeOfTransport.barge: 0,
        ModeOfTransport.feeder: 0.8 / (0.8 + 4.6) + diff,
        ModeOfTransport.deep_sea_vessel: 4.6 / (0.8 + 4.6) - diff
    },
    ModeOfTransport.barge: {
        ModeOfTransport.truck: 0,
        ModeOfTransport.train: 0,
        ModeOfTransport.barge: 0,
        ModeOfTransport.feeder: 0.8 / (0.8 + 4.6) + diff,
        ModeOfTransport.deep_sea_vessel: 4.6 / (0.8 + 4.6) - diff
    },
    ModeOfTransport.feeder: {
        ModeOfTransport.truck: 0.8 / (0.8 + 1.9) * 0.502,
        ModeOfTransport.train: 0.8 / (0.8 + 1.9) * 0.47,
        ModeOfTransport.barge: 0.8 / (0.8 + 1.9) * 0.0028,
        ModeOfTransport.feeder: 0,
        ModeOfTransport.deep_sea_vessel: 1.9 / (0.8 + 1.9)
    },
    ModeOfTransport.deep_sea_vessel: {
        ModeOfTransport.truck: 4.6 / (4.6 + 1.9) * 0.502,
        ModeOfTransport.train: 4.6 / (4.6 + 1.9) * 0.47,
        ModeOfTransport.barge: 4.6 / (4.6 + 1.9) * 0.0028,
        ModeOfTransport.feeder: 1.9 / (4.6 + 1.9),
        ModeOfTransport.deep_sea_vessel: 0
    }
}

ModeOfTransportDistributionManager().set_mode_of_transport_distributions(mode_of_transport_distribution_1c)

container_flow_generation_manager.generate()
export_container_flow_manager = ExportContainerFlowManager()
folder_data_visual_1c = "demo-DEHAM-visual-1c-0-1--" + str(datetime.datetime.now()).replace(":", "-").replace(" ", "--").split(".")[0]
export_container_flow_manager.export(
    folder_name=folder_data_visual_1c,
    file_format=ExportFileFormat.csv
)

database_chooser.close_current_connection()

## Prepare visuals

Later we will convert SVG files to EMF files.
While Microsoft does not support SVG files in its Office suite, EMF is one option how to include vector graphics into Word or PowerPoint.
One way to convert from SVG to EMF is by using the open source software Inkscape.

In [ ]:
if os.name == 'nt':
    path_to_inkscape_executable = r"C:\Program Files\Inkscape\bin\inkscape.exe"
else:
    raise RuntimeException("Add your path to your inkscape executable here")


def convert_to_emf(input_file):
    output_file = "".join(input_file.split(".")[0:-1]) + ".emf"
    !"{path_to_inkscape_executable}" "{input_file}" --export-filename "{output_file}"

## Load data

In [ ]:
export_folder = os.path.abspath(
    os.path.join(
        os.path.dirname(conflowgen.__path__[0]),
        "conflowgen",
        "data",
        "exports"
    )
)

path_to_folder_data_visual_1a = os.path.join(
    export_folder,
    folder_data_visual_1a
)

path_to_folder_data_visual_1b = os.path.join(
    export_folder,
    folder_data_visual_1b
)

path_to_folder_data_visual_1c = os.path.join(
    export_folder,
    folder_data_visual_1c
)

print("Working with: ", (path_to_folder_data_visual_1a, path_to_folder_data_visual_1b, path_to_folder_data_visual_1c))

### Load containers

In [ ]:
def load_containers(path_to_folder_data: str):

    path_to_containers = os.path.join(
        path_to_folder_data,
        "containers.csv"
    )
    df_containers = pd.read_csv(path_to_containers, index_col="id", dtype={
        "delivered_by_truck": "Int64",
        "picked_up_by_truck": "Int64",
        "delivered_by_large_scheduled_vehicle": "Int64",
        "picked_up_by_large_scheduled_vehicle": "Int64"
    })
    return df_containers


df_containers_1a = load_containers(path_to_folder_data_visual_1a)
df_containers_1b = load_containers(path_to_folder_data_visual_1b)
df_containers_1c = load_containers(path_to_folder_data_visual_1c)

In [ ]:
df_containers_1b.groupby(by="delivered_by_large_scheduled_vehicle").count()

### Load vehicles adhering to a schedule

In [ ]:
def get_scheduled_vehicle_file_paths(path_to_folder_data_visual: str):
    path_to_deep_sea_vessels = os.path.join(
        path_to_folder_data_visual,
        "deep_sea_vessels.csv"
    )

    path_to_feeders = os.path.join(
        path_to_folder_data_visual,
        "feeders.csv"
    )

    path_to_barges = os.path.join(
        path_to_folder_data_visual,
        "barges.csv"
    )

    path_to_trains = os.path.join(
        path_to_folder_data_visual,
        "trains.csv"
    )

    scheduled_vehicle_file_paths = {
        "deep_sea_vessels": path_to_deep_sea_vessels,
        "feeders": path_to_feeders,
        "barges": path_to_barges,
        "trains": path_to_trains
    }

    for name, path in scheduled_vehicle_file_paths.items():
        print("Check file exists for vehicle " + name + f" in folder {path_to_folder_data_visual}.")
        assert os.path.isfile(path)

    return scheduled_vehicle_file_paths


scheduled_vehicle_file_paths_1a = get_scheduled_vehicle_file_paths(path_to_folder_data_visual_1a)
scheduled_vehicle_file_paths_1b = get_scheduled_vehicle_file_paths(path_to_folder_data_visual_1b)
scheduled_vehicle_file_paths_1c = get_scheduled_vehicle_file_paths(path_to_folder_data_visual_1c)

In [ ]:
for scheduled_vehicle_file_paths in (scheduled_vehicle_file_paths_1a, scheduled_vehicle_file_paths_1b, scheduled_vehicle_file_paths_1c):
    for name, path in list(scheduled_vehicle_file_paths.items()):
        print("Check file size for vehicle " + name + f" in folder {path}")
        size_in_bytes = os.path.getsize(path)
        if size_in_bytes <= 4:
            print("    This file is empty, ignoring it in the analysis from now on")
            del scheduled_vehicle_file_paths_1b[name]
        else:
            print("Everything is ok")

In [ ]:
def get_scheduled_vehicle_dfs(scheduled_vehicle_file_paths, figure_name: str):
    scheduled_vehicle_dfs = {
        name: pd.read_csv(path, index_col=0, parse_dates=["scheduled_arrival"])
        for name, path in scheduled_vehicle_file_paths.items()
    }

    for name, df in scheduled_vehicle_dfs.items():
        display(Markdown("#### " + name.replace("_", " ").capitalize() + " " + figure_name))
        scheduled_vehicle_dfs[name]["vehicle_type"] = name
        display(scheduled_vehicle_dfs[name].sort_values(by="scheduled_arrival"))
    return scheduled_vehicle_dfs


scheduled_vehicle_dfs_1a = get_scheduled_vehicle_dfs(scheduled_vehicle_file_paths_1a, "1a")
scheduled_vehicle_dfs_1b = get_scheduled_vehicle_dfs(scheduled_vehicle_file_paths_1b, "1b")
scheduled_vehicle_dfs_1c = get_scheduled_vehicle_dfs(scheduled_vehicle_file_paths_1c, "1c")

In [ ]:
def prepare_df_large_scheduled_vehicle(scheduled_vehicle_dfs):
    df_large_scheduled_vehicle = pd.concat(
        scheduled_vehicle_dfs.values()
    )
    df_large_scheduled_vehicle.sort_index(inplace=True)
    df_large_scheduled_vehicle.info()
    return df_large_scheduled_vehicle


df_large_scheduled_vehicle_1a = prepare_df_large_scheduled_vehicle(scheduled_vehicle_dfs_1a)
df_large_scheduled_vehicle_1b = prepare_df_large_scheduled_vehicle(scheduled_vehicle_dfs_1b)
df_large_scheduled_vehicle_1c = prepare_df_large_scheduled_vehicle(scheduled_vehicle_dfs_1c)

df_large_scheduled_vehicle_1b

### Load trucks

In [ ]:
path_to_trucks_1a = os.path.join(
    path_to_folder_data_visual_1a,
    "trucks.csv"
)
assert os.path.isfile(path_to_trucks_1a)
path_to_trucks_1b = os.path.join(
    path_to_folder_data_visual_1b,
    "trucks.csv"
)
assert os.path.isfile(path_to_trucks_1b)
path_to_trucks_1c = os.path.join(
    path_to_folder_data_visual_1c,
    "trucks.csv"
)
assert os.path.isfile(path_to_trucks_1c)

In [ ]:
def load_trucks(path: str):
    return pd.read_csv(
        path_to_trucks_1b, index_col=0,
        parse_dates=[
            # Pickup
            "planned_container_pickup_time_prior_berthing",
            "realized_container_pickup_time",

            # Delivery
            "planned_container_delivery_time_at_window_start",
            "realized_container_delivery_time"
        ])


df_truck_1a = load_trucks(path_to_trucks_1a)
df_truck_1b = load_trucks(path_to_trucks_1b)
df_truck_1c = load_trucks(path_to_trucks_1c)

df_truck_1b

## Plot arrival distribution over time

Prepare data

In [ ]:
x, y, z = [], [], []
y_axis = []

y_scaling_factor = 2

for i, (name, df) in enumerate(scheduled_vehicle_dfs_1b.items()):
    y_axis.append((i/y_scaling_factor, name))
    if len(df) == 0:
        continue
    for _, row in df.iterrows():
        if row["vehicle_type"] == "trains":
            continue
        event = row["scheduled_arrival"]
        moved_capacity = row["moved_capacity"]
        x.append(event)
        y.append(i / y_scaling_factor)
        z.append(moved_capacity)


arrivals_and_capacity_1b = pd.DataFrame({"x": x, "y": y, "z": z})
display(arrivals_and_capacity_1b)
arrivals_and_capacity_1b.y.unique()

In [ ]:
container_deliveries_by_truck_1b = df_truck_1b.groupby(
    pd.Grouper(key='realized_container_delivery_time', freq='H')
).count().fillna(0)

Create figure that shows the arrivals of different vehicles next to each other.
This creates a first impression of the ramp-up and ramp-down phase.

In [ ]:
fig = plt.figure(figsize=(10, 5))

gs = gridspec.GridSpec(2, 1, height_ratios=[2, 1])

ax1 = plt.subplot(gs[0])  # the upper subplot

ax1.plot(container_deliveries_by_truck_1b["delivers_container"], color="k", linewidth=.7)
ax1.set_ylabel("Truck arrivals per hour")
ax1.set_xlabel("")
ax1.set_xlim([pd.Timestamp("2021-06-27"), pd.Timestamp(pd.Timestamp("2021-08-02"))])

ax2 = plt.subplot(gs[1], sharex=ax1)  # the lower subplot

color_capacity_dots = 'dimgray'

# x-axis: time, y-axis: dummy variable for lower or upper row
moved_teu_scale_factor = 10
scatterplot = ax2.scatter(
    x, y,
    s=np.array(z)/moved_teu_scale_factor,
    marker="o", color=color_capacity_dots, edgecolor="k"
)

myFmt = mdates.DateFormatter('%d.%m.')
ax1.xaxis.set_major_formatter(myFmt)
ax2.xaxis.set_major_formatter(myFmt)
ax2.set_yticks([0.0, .5])
ax2.set_yticklabels(["Deep sea\nvessels", "Feeders"])
ax2.set_ylim([-0.3, 0.7])

handles, labels = scatterplot.legend_elements(prop="sizes", num=4)
adjusted_labels = []
for handle, label in zip(handles, labels):
    label_digits_only = "".join([c for c in label if c in string.digits])
    number = int(label_digits_only)
    number *= moved_teu_scale_factor
    formatted_label = "$\\mathdefault{" + str(number) + "}$"
    adjusted_labels.append(formatted_label)
    handle.set_markerfacecolor(color_capacity_dots)

ax2.legend(
    handles,
    adjusted_labels,
    borderpad=1.2,
    labelspacing=1.2,
    loc=(1.02, 0),
    title="Moved TEU"
)

# Hide first date (it is in June) as it distorts the plot
ax2.set_xticks(ax2.get_xticks()[1:])

# just comment this in in case you want to save the figure
plt.savefig("relationship_truck_deliveries_and_vessel_departures.svg", bbox_inches='tight')

plt.show()

In [ ]:
# just comment this in in case you want to convert the figure from svg to emf
convert_to_emf("relationship_truck_deliveries_and_vessel_departures.svg")

## Plot capacity for inbound and outbound movements

Prepare data

In [ ]:
def add_delivery_and_pickup(df_containers: pd.DataFrame, df_large_scheduled_vehicle: pd.DataFrame):
    vehicle_to_teu_to_deliver = {}
    vehicle_to_teu_to_pickup = {}

    for i, container in df_containers.iterrows():
        teu = container["length"] / 20
        assert 1 <= teu <= 2.5

        if container["delivered_by"] != "truck":
            vehicle = container["delivered_by_large_scheduled_vehicle"]
            if vehicle not in vehicle_to_teu_to_deliver.keys():
                vehicle_to_teu_to_deliver[vehicle] = 0
            vehicle_to_teu_to_deliver[vehicle] += teu

        if container["picked_up_by"] != "truck":
            vehicle = container["picked_up_by_large_scheduled_vehicle"]
            if vehicle not in vehicle_to_teu_to_pickup.keys():
                vehicle_to_teu_to_pickup[vehicle] = 0
            vehicle_to_teu_to_pickup[vehicle] += teu

    s_delivery = pd.Series(vehicle_to_teu_to_deliver)
    s_pickup = pd.Series(vehicle_to_teu_to_pickup)
    df_large_scheduled_vehicle["capacity_delivery"] = s_delivery
    df_large_scheduled_vehicle["capacity_pickup"] = s_pickup


add_delivery_and_pickup(df_containers_1a, df_large_scheduled_vehicle_1a)
add_delivery_and_pickup(df_containers_1b, df_large_scheduled_vehicle_1b)
add_delivery_and_pickup(df_containers_1c, df_large_scheduled_vehicle_1c)
df_large_scheduled_vehicle_1b

Compare 1a, 1b, and 1c with each other.
This is an example of how parameters need to be tuned.

In [ ]:
fig, axs = plt.subplots(
    nrows=1,
    ncols=3,
    figsize=(10, 3.3),
    sharey=True
)

colors = {
    'feeders': 'navy',
    'deep_sea_vessels': 'olive'
}

for title, ax, df_large_scheduled_vehicle in zip("abc", axs, (
    df_large_scheduled_vehicle_1a,
    df_large_scheduled_vehicle_1b,
    df_large_scheduled_vehicle_1c
)):
    df_large_scheduled_vehicle = df_large_scheduled_vehicle[
        ~df_large_scheduled_vehicle['vehicle_type'].isin(
            [
                "trains",
                "barges"
            ]
        )
    ]
    ax.scatter(
        df_large_scheduled_vehicle['capacity_delivery'],
        df_large_scheduled_vehicle['capacity_pickup'],
        c=list(df_large_scheduled_vehicle['vehicle_type'].map(colors)),
        marker="."
    )
    ax.set_title(title)
    ax.axline((0, 0), slope=1.2, color='dimgray', label='Ratio of 1:1.2 (20% buffer)')
    ax.axline((0, 0), slope=1,   color='lightgray', label='Ratio of 1:1')
    ax.set_xlim([0, 3500])
    ax.set_ylim([0, 3500])
    ax.set_aspect('equal', adjustable='box')
    ax.grid(color='lightgray', linestyle=':', linewidth=.5)

plt.setp(axs[:], xlabel="TEU delivered by vessel")
axs[0].set_ylabel("TEU picked up by vessel")

handles, labels = ax.get_legend_handles_labels()
for vehicle_type, color in colors.items():
    vehicle_type_legend_name = vehicle_type.replace("_", " ").capitalize()
    patch = plt.Line2D([], [], color=color, marker="o", linewidth=0, label=vehicle_type_legend_name)
    handles.append(patch)

plt.legend(
    handles=handles,
    loc='lower left',
    bbox_to_anchor=(-2.2, -0.4),
    fancybox=True,
    ncol=4
)

# just comment this in in case you want to save the figure
plt.savefig("ratio_delivered_and_picked_up_containers_comparison.svg", bbox_inches='tight')
plt.show()

In [ ]:
# just comment this in in case you want to convert the figure from svg to emf
convert_to_emf("ratio_delivered_and_picked_up_containers_comparison.svg")